In [3]:
import numpy as np
import pandas as pd
import os
import glob
import csv

In [2]:
def parse_out(filename):
    metaG = str()
    ags = int()
    bp = int()
    genome_eq = int()
    reads = int()
    infile = open(filename, 'r')
    file=[]
    for Line in infile:
        Line = Line.strip("\n")
        Line = Line.split("\t")
        file.append(Line)
    for elem in file:
        if elem[0] == 'metagenome:':
            metaG = elem[1].split(".")[0]
        if elem[0] == 'average_genome_size:':
            ags = elem[1]
        if elem[0] == 'total_bases:':
            bp = elem[1]
        if elem[0] == 'genome_equivalents:':
            genome_eq = elem[1]
        if elem[0] == 'reads_sampled:':
            reads = elem[1]
        else:
            next
    #returnlist = [metaG, ags, bp, genome_eq]
    return metaG, ags, bp, genome_eq, reads



In [3]:
os.chdir("../../../data/input/Raw_bioinfo/")
census_files = glob.glob("*census.txt")

In [4]:
census_data = pd.DataFrame(columns=["ID", "AGS", "bp", "Genomes", "reads"])

In [5]:
i = 0
for f in census_files:
    i += 1
    census_data.loc[i] = list(parse_out(f))
    

In [6]:
census_data.ID = census_data.ID.str.replace("_RPE_1","")
census_data

,ID,AGS,bp,Genomes,reads
1,BMI_Plate10WellA2_mms,6113683.90291,2188103613,357.902640658,2000000
2,BMI_Plate81WellG4,5893983.17843,3000016122,508.996383461,2000000
3,BMI_Plate81WellF1,8574827.07887,2172643387,253.374600679,2000000
4,BMI_Plate72WellG4,7950295.40616,4782868115,601.596276698,2000000
5,BMI_Plate72WellF1,5889560.87872,2970681281,504.397754293,2000000
...,...,...,...,...,...
394,BMI_Plate41WellC2,5972941.03317,8017468147,1342.29822502,2000000
395,BMI_Plate41WellB7,5740209.15808,3889599213,677.605833844,2000000
396,BMI_Plate19WellB5_mms,8188647.91804,1567855896,191.467005505,2000000
397,BMI_Plate88WellG3,6358245.94996,1784753779,280.699078495,2000000


In [8]:
def parse_codon_output(filename):
    skip = 0
    output_line = []
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if skip == 0:
                skip += 1
            else:
                output_line.append(row[1])
        output_line.append(filename.replace("_R_genes_AA_codon_output.csv",""))
    return output_line



In [9]:
codon_files = glob.glob("*codon_output.csv")

In [10]:
skip = 0
colnames = []
test_codon_file = codon_files[0]
with open(test_codon_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if skip == 0:
            skip += 1
        else:
            colnames.append(row[0])

colnames.append("ID")
Codon_DF = pd.DataFrame(columns=colnames)

In [11]:
row = 0
for f in codon_files:
    Codon_DF.loc[row] = parse_codon_output(f)
    row += 1
    

In [12]:
Codon_DF.to_csv("../derived/codon_usage.csv")

In [13]:
gc_files = glob.glob("*GC_output.txt")

In [14]:
skip = 0
with open(gc_files[0]) as gc_test_file:
    for line in gc_test_file:
        if skip == 0:
            colnames = line.strip("\n").split("\t")
            skip += 1
        else:
            next

colnames[0] = "ID"

GC_DF = pd.DataFrame(columns=colnames)


In [15]:
def GC_parse_output(filename):
    skip = 0
    gc_output_line = []
    with open(filename) as gc_test_file:
        for line in gc_test_file:
            if skip == 0:
                skip += 1
            else:
                gc_output_line = line.strip("\n").split("\t")
    return gc_output_line



In [16]:
row = 0
for f in gc_files:
    GC_DF.loc[row] = GC_parse_output(f)
    row += 1

In [17]:
GC_DF.ID = GC_DF.ID.str.replace("_R_genes","")

In [18]:
census_gc = census_data.merge(GC_DF, on=['ID'], how = 'left')

In [19]:
census_gc = census_gc.dropna()
census_gc["AGS"] = pd.to_numeric(census_gc["AGS"])
census_gc["GC_bact"] = pd.to_numeric(census_gc["GC_bact"])
census_gc["AA_CN"] = pd.to_numeric(census_gc["AA_CN"])
census_gc["GC"] = pd.to_numeric(census_gc["GC"])

In [20]:
census_gc.to_csv("../derived/genomic_traits.csv")